# 🚀 Módulo 2.5: Métodos de Ensamblado - Boosting
### Curso: **Machine Learning con Python** (IFCD093PO)
**Duración estimada:** 10 horas

---

## 🎯 Objetivos del Módulo

Si Random Forest es un comité de expertos independientes, el Boosting es una línea de montaje donde cada experto aprende de los errores del anterior. En este módulo, dominarás el **Boosting**, la otra gran familia de métodos de ensamblado, conocida por su increíble rendimiento y por ser la opción preferida en competiciones de Machine Learning.

Al finalizar, serás capaz de:

- ✅ Entender el concepto de **aprendizaje secuencial** y cómo los modelos de boosting se construyen para corregir errores pasados.
- ✅ Comprender el funcionamiento de **AdaBoost** y **Gradient Boosting**, los pilares del boosting.
- ✅ Implementar y optimizar los algoritmos de vanguardia: **XGBoost** y **LightGBM**.
- ✅ Conocer las ventajas clave de XGBoost y LightGBM, como la velocidad, la regularización y el manejo de datos faltantes.
- ✅ Distinguir claramente entre las filosofías de **Bagging (Random Forest)** y **Boosting**.

**¡Prepárate para construir modelos que aprenden de sus errores para alcanzar la excelencia!**

---

## 📚 Tabla de Contenidos

1. [Boosting: Aprendizaje Secuencial](#1-boosting)
   - [La Idea: Corregir Errores Pasados](#1.1-idea)
2. [AdaBoost (Adaptive Boosting)](#2-adaboost)
   - [Poniendo Foco en los Casos Difíciles](#2.1-foco)
   - [Implementación de AdaBoost](#2.2-implementacion-ada)
3. [Gradient Boosting](#3-gradient-boosting)
   - [Aprendiendo de los Errores Residuales](#3.1-residuos)
   - [Implementación de Gradient Boosting](#3.2-implementacion-gb)
4. [XGBoost: El Rey de las Competiciones](#4-xgboost)
   - [¿Por qué es tan Popular?](#4.1-popular)
   - [Implementación de XGBoost](#4.2-implementacion-xgb)
5. [LightGBM: Velocidad y Eficiencia](#5-lightgbm)
   - [Crecimiento por Hoja (Leaf-wise)](#5.1-leaf-wise)
   - [Implementación de LightGBM](#5.2-implementacion-lgbm)
6. [Comparativa Final: Bagging vs. Boosting](#6-comparativa)
7. [Resumen y Próximos Pasos](#7-resumen)

---

## 🏃‍♂️ 1. Boosting: Aprendizaje Secuencial <a id='1-boosting'></a>

### 1.1 La Idea: Corregir Errores Pasados <a id='1.1-idea'></a>

A diferencia del Bagging (Random Forest) donde los árboles se entrenan en paralelo y de forma independiente, el **Boosting** es un método **secuencial**.

**El proceso es el siguiente:**
1.  Se entrena un primer modelo simple (un "aprendiz débil", a menudo un árbol de decisión muy corto).
2.  Se evalúan las predicciones de este modelo y se identifican sus errores.
3.  Se entrena un **segundo modelo** que se enfoca específicamente en **corregir los errores** del primer modelo.
4.  Se entrena un tercer modelo que intenta corregir los errores combinados de los dos primeros.
5.  Este proceso se repite `n` veces (el número de estimadores).

La predicción final es una **suma ponderada** de las predicciones de todos los modelos, donde los modelos que funcionaron mejor tienen más peso.

![Boosting](imagenes/Boosting.png)

## 💪 2. AdaBoost (Adaptive Boosting) <a id='2-adaboost'></a>

### 2.1 Poniendo Foco en los Casos Difíciles <a id='2.1-foco'></a>

AdaBoost fue uno de los primeros algoritmos de boosting y su idea es muy intuitiva.

Funciona ajustando el **peso de las muestras de entrenamiento**. Inicialmente, todas las muestras tienen el mismo peso. Después de entrenar un aprendiz, AdaBoost **aumenta el peso** de las muestras que fueron **mal clasificadas**. En la siguiente iteración, el nuevo aprendiz se verá forzado a prestar más atención a estos casos difíciles.

Esto hace que el sistema se adapte (de ahí "Adaptive") y se enfoque en las partes más complejas del problema.

In [ ]:
from sklearn.ensemble import AdaBoostClassifier ## Importar AdaBoost
from sklearn.tree import DecisionTreeClassifier ## Importar árbol de decisión
from sklearn.datasets import make_moons ## Importar conjunto de datos de lunas
from sklearn.model_selection import train_test_split ## Importar función para dividir datos
from sklearn.metrics import accuracy_score ## Importar métrica de precisión
# Eliminamos las advertencias para mayor claridad
import warnings
warnings.filterwarnings("ignore")
# Crear conjunto de datos de lunas
X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# AdaBoost con 30 árboles de decisión de profundidad 1 (stumps)
# Cada árbol contribuye con un learning_rate de 0.5
# n_estimators define el número de árboles en el ensemble
# algorithm define el método de actualización de pesos
# algorithm="SAMME" es menos eficiente pero puede ser útil en algunos casos
# otros valores posibles son "SAMME.R", que es más eficiente para clasificación binaria
# El parámetro algorithm de AdaBoostClassifier solo acepta 'SAMME' 
# como un valor válido cuando se utiliza un estimador base 
# personalizado (como DecisionTreeClassifier). 
# El valor predeterminado 'SAMME.R' solo es compatible 
# cuando el estimador base admite estimaciones de probabilidad, 
# lo que no siempre es el caso para estimadores personalizados 
# en algunas versiones de scikit-learn.
ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1),
    n_estimators=30,
    algorithm="SAMME",
    learning_rate=0.5, # El learning_rate controla cuánto contribuye cada árbol
    random_state=42
)

# Entrenar y evaluar el modelo AdaBoost
ada_clf.fit(X_train, y_train)
y_pred_ada = ada_clf.predict(X_test)

print(f"Accuracy de AdaBoost: {accuracy_score(y_test, y_pred_ada):.4f}")

Accuracy de AdaBoost: 0.8800


## 📉 3. Gradient Boosting <a id='3-gradient-boosting'></a>

### 3.1 Aprendiendo de los Errores Residuales <a id='3.1-residuos'></a>

Gradient Boosting lleva la idea del boosting un paso más allá. En lugar de ajustar los pesos de las muestras, intenta ajustar los **errores residuales** del predictor anterior.

**Proceso (para regresión):**
1.  Se entrena un primer árbol de regresión simple sobre los datos.
2.  Se calculan los **errores (residuales)**: `error = valor_real - predicción`.
3.  Se entrena un **segundo árbol** para que **prediga esos errores**.
4.  La nueva predicción combinada es `predicción_arbol_1 + learning_rate * predicción_arbol_2`.
5.  Se calculan los nuevos residuales y se repite el proceso.

El nombre "Gradient" viene de que este proceso es una forma de optimización por descenso de gradiente en el espacio de las funciones. Es un método extremadamente potente.

In [4]:
from sklearn.ensemble import GradientBoostingRegressor ## Importar Gradient Boosting Regressor
from sklearn.metrics import mean_squared_error ## Importar métrica de error cuadrático medio
import numpy as np ## Importar NumPy

# Datos de ejemplo para regresión, creando una función no lineal con ruido
np.random.seed(42)
X_reg = np.random.rand(100, 1) * 10
y_reg = np.sin(X_reg).ravel() + np.random.randn(100) * 0.5
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(X_reg, y_reg, random_state=42)

# Entrenar un Gradient Boosting Regressor
gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=120, learning_rate=0.1, random_state=42)
gbrt.fit(X_train_r, y_train_r)

y_pred_gbrt = gbrt.predict(X_test_r)
rmse = np.sqrt(mean_squared_error(y_test_r, y_pred_gbrt))
print(f"RMSE de Gradient Boosting: {rmse:.4f}")

RMSE de Gradient Boosting: 0.4216


---

## 👑 4. XGBoost: El Rey de las Competiciones <a id='4-xgboost'></a>

**XGBoost (eXtreme Gradient Boosting)** es una implementación optimizada y mejorada de Gradient Boosting. Durante años, ha sido el algoritmo dominante en competiciones de Machine Learning como Kaggle.

### 4.1 ¿Por qué es tan Popular? <a id='4.1-popular'></a>

1.  **Velocidad y Rendimiento**: Está implementado en C++ y es extremadamente rápido. Puede entrenar en paralelo (a nivel de características).
2.  **Regularización Incorporada**: Incluye regularización L1 (Lasso) y L2 (Ridge) en su función de coste, lo que lo hace muy robusto contra el overfitting.
3.  **Manejo de Datos Faltantes**: Puede manejar valores nulos de forma nativa.
4.  **Validación Cruzada Integrada**: Puede realizar validación cruzada en cada iteración.
5.  **Poda de Árboles Avanzada**: Realiza una poda más inteligente que el Gradient Boosting estándar.

XGBoost no está incluido en Scikit-learn por defecto, por lo que necesita ser instalado.

In [5]:
# Para instalar XGBoost, ejecuta en tu terminal:
# pip install xgboost

import xgboost # Importar XGBoost una vez instalado

# Entrenar un clasificador XGBoost
# Los parametros son:
# random_state para reproducibilidad
# use_label_encoder=False para evitar advertencias sobre el codificador de etiquetas
# eval_metric='logloss' para definir la métrica de evaluación
xgb_clf = xgboost.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train, y_train)

y_pred_xgb = xgb_clf.predict(X_test)
print(f"Accuracy de XGBoost: {accuracy_score(y_test, y_pred_xgb):.4f}")

Accuracy de XGBoost: 0.8640


## 💡 5. LightGBM: Velocidad y Eficiencia <a id='5-lightgbm'></a>

**LightGBM (Light Gradient Boosting Machine)** es otro framework de boosting desarrollado por Microsoft. Su principal ventaja es su **velocidad y eficiencia en datasets muy grandes**.

### 5.1 Crecimiento por Hoja (Leaf-wise) <a id='5.1-leaf-wise'></a>

La principal diferencia con XGBoost es cómo crecen los árboles:
- **XGBoost** crece por **nivel (level-wise)**: Completa un nivel entero del árbol antes de pasar al siguiente. Es más sistemático.
- **LightGBM** crece por **hoja (leaf-wise)**: Expande la hoja que más reduce la pérdida, lo que lleva a árboles más asimétricos pero a menudo más eficientes.

![Leaf-wise](imagenes/Leaf-wise.png)

Esto hace que LightGBM sea extremadamente rápido, aunque a veces puede ser más propenso al overfitting en datasets pequeños.

In [6]:
# Para instalar LightGBM, ejecuta en tu terminal:
# pip install lightgbm

import lightgbm

lgbm_clf = lightgbm.LGBMClassifier(random_state=42)
lgbm_clf.fit(X_train, y_train)

y_pred_lgbm = lgbm_clf.predict(X_test)
print(f"Accuracy de LightGBM: {accuracy_score(y_test, y_pred_lgbm):.4f}")

[LightGBM] [Info] Number of positive: 186, number of negative: 189
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000075 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 250
[LightGBM] [Info] Number of data points in the train set: 375, number of used features: 2
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.496000 -> initscore=-0.016000
[LightGBM] [Info] Start training from score -0.016000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


Esta salida de LightGBM muestra información y advertencias del proceso de entrenamiento. Lo más importante es la advertencia repetida:

*[LightGBM] [Warning] No further splits with positive gain, best gain: -inf*

Esto significa que, para los datos y parámetros usados, LightGBM no encuentra ninguna división (split) en los árboles que mejore la función objetivo (ganancia positiva). Por eso, los árboles que construye son muy simples (a veces solo la raíz).

Las causas más comunes son:

- **Muy pocas variables/features:** En tu salida, dice "number of used features: 2". Con solo 2 variables, es fácil que no haya splits útiles.
- **Datos poco variados o mal preparados:** Si las variables no tienen suficiente variabilidad o no están bien escaladas/categorizadas, el modelo no puede aprender.
- **Parámetros restrictivos:** Si usas un valor muy bajo de max_depth, min_data_in_leaf alto, o min_gain_to_split alto, puede impedir que se hagan splits.

**¿Es un error grave?**
No es un error crítico si el accuracy es bueno, pero indica que el modelo no está aprovechando la estructura de los datos. Si ves muchas advertencias y el accuracy es bajo, revisa tus datos y parámetros.

**¿Qué hacer?**

- Asegúrate de tener suficientes variables predictoras.
- Revisa la preparación de los datos.
- Ajusta los hiperparámetros (por ejemplo, baja min_gain_to_split, baja min_data_in_leaf, aumenta max_depth).
- Si solo tienes 2 variables, prueba añadiendo más o usando otro modelo.

---

## ⚖️ 6. Comparativa Final: Bagging vs. Boosting <a id='6-comparativa'></a>

| Característica | Bagging (Random Forest) | Boosting (Gradient Boosting, XGBoost) |
| :--- | :--- | :--- |
| **Filosofía** | Entrena modelos **independientes** en paralelo. | Entrena modelos **secuencialmente**, aprendiendo de errores. |
| **Objetivo Principal** | Reducir la **varianza** (overfitting). | Reducir el **sesgo** (underfitting) y la varianza. |
| **Rendimiento** | Muy bueno y robusto. | A menudo, el **mejor rendimiento** posible (state-of-the-art). |
| **Sensibilidad** | Menos sensible a los hiperparámetros. | Más sensible a los hiperparámetros (`learning_rate`, etc.). |
| **Velocidad** | Puede ser paralelizado fácilmente. | Es secuencial, pero XGBoost/LightGBM son muy rápidos. |
| **Primer Intento** | Excelente como primer modelo a probar. | Excelente para exprimir el máximo rendimiento. |

---

## 📝 7. Resumen y Próximos Pasos <a id='7-resumen'></a>

### 🎉 ¡Has llegado a la cima del Aprendizaje Supervisado!

#### ✅ Lo que has aprendido:

1. **Filosofía del Boosting**
   - El poder del **aprendizaje secuencial**, donde cada modelo se enfoca en corregir los errores de los anteriores.

2. **Algoritmos Clave**
   - **AdaBoost**, que ajusta los pesos de las muestras para centrarse en los casos difíciles.
   - **Gradient Boosting**, que entrena nuevos modelos para predecir los errores residuales de los anteriores.

3. **Implementaciones de Vanguardia**
   - **XGBoost** y **LightGBM**, las librerías optimizadas que dominan el Machine Learning práctico por su velocidad, regularización y rendimiento superior.

4. **Bagging vs. Boosting**
   - La diferencia fundamental entre reducir la varianza (Bagging) y reducir el sesgo (Boosting).

---

### 🚀 Próximo Módulo: Introducción al Aprendizaje No Supervisado

Has completado un viaje exhaustivo por el Aprendizaje Supervisado. Ahora, es el momento de cambiar de paradigma. ¿Qué hacemos cuando **no tenemos etiquetas**? ¿Cuándo no hay una "respuesta correcta" que aprender?

En el próximo módulo, nos adentraremos en el **Aprendizaje No Supervisado**, donde el objetivo es encontrar patrones, estructura y conocimiento oculto en los datos por sí mismos. Empezaremos con una introducción a este nuevo y emocionante campo.

**Has aprendido a predecir el futuro basándote en el pasado. Ahora, ¡vamos a descubrir la estructura oculta del presente!**